In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer,TrainingArguments, Trainer, BertForSequenceClassification
from peft import LoraConfig, TaskType, get_peft_model, AutoPeftModelForSequenceClassification
import numpy as np
import evaluate
import torch

C:\Users\stefa\anaconda3\envs\cuda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Load Dataset IMDB - Movie DataBase
dataset = load_dataset("imdb")

In [3]:
#Load tokenizer and tokenize dataset
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [4]:
#Dataset split
LEN_TRAIN = 5000 #len(tokenized_datasets["train"])
LEN_TEST = 1500 #len(tokenized_datasets["test"])
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(LEN_TRAIN))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(LEN_TEST))

In [5]:
# Create and apply PEFT config
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

In [6]:
#load pretrained BERT
model = BertForSequenceClassification.from_pretrained(
    'bert-base-cased', 
    num_labels=2
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Apply PEFT and LORA
model = get_peft_model(model, lora_config)

In [8]:
# Define evalute metric
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
training_args = TrainingArguments(
        output_dir="./peft_model_output", 
        eval_strategy="epoch",
        num_train_epochs=12
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [11]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# To make sure the model uses the GPU:
trainer.model.to(device)

Using device: cuda


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): BertForSequenceClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(28996, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0-11): 12 x BertLayer(
              (attention): BertAttention(
                (self): BertSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default

In [12]:
eval_results_base = trainer.evaluate(small_eval_dataset)
print("Pretrained Model Accuracy:", eval_results_base)

Pretrained Model Accuracy: {'eval_loss': 0.7065376043319702, 'eval_model_preparation_time': 0.0063, 'eval_accuracy': 0.45866666666666667, 'eval_runtime': 6.249, 'eval_samples_per_second': 240.038, 'eval_steps_per_second': 30.085}


In [13]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy
1,0.690400,0.630910,0.006300,0.650000
2,0.603200,0.423721,0.006300,0.804667
3,0.438300,0.343960,0.006300,0.859333
4,0.338100,0.326415,0.006300,0.864667
5,0.319400,0.315533,0.006300,0.874000
6,0.319100,0.311139,0.006300,0.877333
7,0.309700,0.314885,0.006300,0.881333
8,0.301400,0.300586,0.006300,0.882000
9,0.292700,0.299603,0.006300,0.882000
10,0.289500,0.296611,0.006300,0.882667


TrainOutput(global_step=7500, training_loss=0.3642292195638021, metrics={'train_runtime': 662.858, 'train_samples_per_second': 90.517, 'train_steps_per_second': 11.315, 'total_flos': 1.579374157824e+16, 'train_loss': 0.3642292195638021, 'epoch': 12.0})

In [14]:
# Save the fine-tuned PEFT model
model.save_pretrained("./peft_finetuned_save")

In [15]:
#Load the fine-tuned PEFT model and tokenizer
model_name_saved = "./peft_finetuned_save"  # Path to your fine-tuned PEFT model (could be a local path or from Hugging Face Hub)
model_infer = AutoPeftModelForSequenceClassification.from_pretrained(model_name_saved)
trainer_infer = Trainer(
    model=model_infer,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
eval_results_peft=trainer_infer.evaluate(small_eval_dataset)

In [17]:
print("Base Model Accuracy:", eval_results_base)
print("Fine-Tuned PEFT Model Accuracy:", eval_results_peft)

Base Model Accuracy: {'eval_loss': 0.7065376043319702, 'eval_model_preparation_time': 0.0063, 'eval_accuracy': 0.45866666666666667, 'eval_runtime': 6.249, 'eval_samples_per_second': 240.038, 'eval_steps_per_second': 30.085}
Fine-Tuned PEFT Model Accuracy: {'eval_loss': 0.2977752685546875, 'eval_model_preparation_time': 0.0066, 'eval_accuracy': 0.8846666666666667, 'eval_runtime': 7.5775, 'eval_samples_per_second': 197.956, 'eval_steps_per_second': 24.81}
